In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

stocks = ['AMD', 'NVDA', 'META', 'TSLA']
start = datetime(2018, 1, 1)
end = datetime(2024, 4, 30)

stock_prices = yf.download(stocks, start, end, auto_adjust=False)
stock_prices = stock_prices.resample('ME').last()
stock_prices.index = stock_prices.index.tz_localize(None)
stock_prices = stock_prices.filter(like='Adj Close')
stock_prices.columns = stock_prices.columns.get_level_values(1)

pairs = [('AMD', 'NVDA'), ('AMD', 'META'), ('NVDA', 'TSLA')]
spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = np.log(stock_prices[ticker_1]) - np.log(stock_prices[ticker_2])
    
    spread_mean = spread.expanding().mean()
    spread_std = spread.expanding().std()
    z_score = (spread - spread_mean) / spread_std

    position = np.where(z_score > 0, 'Sell', 'Buy')
    
    pair_df = pd.DataFrame({
        'Date': spread.index,
        'Ticker Pair': f'{ticker_1}-{ticker_2}',
        'Traditional Spread': spread.values,
        'Traditional Z-score': z_score.values,
        'Traditional Position': position
    })

    spread_list.append(pair_df)

full_traditional_df = pd.concat(spread_list)
full_traditional_df.to_csv("outputs/traditional_dummy2.csv", index=False)

[*********************100%***********************]  4 of 4 completed


In [74]:
df = pd.read_csv("traditional_dummy2.csv", parse_dates=["Date"])

returns = pd.read_csv("outputs/returns.csv", parse_dates=["Date"])
merged_df = df.merge(returns, on=["Date", "Ticker Pair"], how="left")

merged_df["Position"] = np.where(
    merged_df["Traditional Position"] == "Buy", 1,
    np.where(merged_df["Traditional Position"] == "Sell", -1, 0)
)

merged_df["Strategy Return"] = merged_df["Position"].shift(1) * merged_df["Return"]

merged_df["Strategy Net Return"] = merged_df["Strategy Return"] - 0.001 * np.abs(merged_df["Position"].diff().fillna(0))

merged_df.to_csv("outputs/traditional_results.csv", index=False)